In [24]:
import numpy as np
from qiskit import QuantumCircuit, transpile, QuantumRegister, ClassicalRegister, execute, BasicAer, Aer
from qiskit.providers.aer import QasmSimulator
from qiskit.visualization import plot_histogram
from qiskit_ibm_runtime import QiskitRuntimeService

# Define Gates

### Reverse CNOT

In [270]:
"""
Input: a, b

Output: 
P = a+b
Q = b
"""

c = QuantumCircuit(2)
c.h(0)
c.h(1)
c.cx(0,1)
c.h(0)
c.h(1)
#c = c.reverse_bits()
revcx = c.to_gate(label="rev_CNOT")
c.draw()

┌───┐     ┌───┐
q_0: ┤ H ├──■──┤ H ├
     ├───┤┌─┴─┐├───┤
q_1: ┤ H ├┤ X ├┤ H ├
     └───┘└───┘└───┘

In [230]:
import qiskit.quantum_info as qi

op = qi.Operator(c)
op

Operator([[1.+0.j, 0.+0.j, 0.+0.j, 0.+0.j],
          [0.+0.j, 0.+0.j, 0.+0.j, 1.+0.j],
          [0.+0.j, 0.+0.j, 1.+0.j, 0.+0.j],
          [0.+0.j, 1.+0.j, 0.+0.j, 0.+0.j]],
         input_dims=(2, 2), output_dims=(2, 2))

In [285]:
def generate_rev_cnot(a,b):
    c = QuantumCircuit(2, 0)
    if (a == '1'):
        c.x(0)
    if (b == '1'):
        c.x(1)
    c.h(0)
    c.h(1)
    c.cx(0,1)
    c.h(0)
    c.h(1)
    c.measure_all()
    c = c.reverse_bits()
    return c

inputs = ['0', '1']
for i in range(len(inputs)):
    for j in range(len(inputs)):
        c = generate_rev_cnot(inputs[i], inputs[j])
        usim = Aer.get_backend('unitary_simulator')
        transpiled = transpile(c, backend=usim)
        backend = Aer.get_backend('aer_simulator')
        job = backend.run(transpiled, shots=1, memory=True)
        output = job.result().get_memory()[0]
        print("in: "+inputs[i]+inputs[j]+" res: "+output)

in: 00 res: 00
in: 01 res: 11
in: 10 res: 10
in: 11 res: 01


In [286]:
c.draw()

┌───┐┌───┐┌───┐┌───┐ ░    ┌─┐
   q_0: ┤ X ├┤ H ├┤ X ├┤ H ├─░────┤M├
        ├───┤├───┤└─┬─┘├───┤ ░ ┌─┐└╥┘
   q_1: ┤ X ├┤ H ├──■──┤ H ├─░─┤M├─╫─
        └───┘└───┘     └───┘ ░ └╥┘ ║ 
meas: 2/════════════════════════╩══╩═
                                1  0

### Peres Gate

In [266]:
"""
Input: a, b, c

Output:
P = a
Q = a + b
R = a*b + c
"""
c = QuantumCircuit(3)
c.ccx(0,1,2)
c.cx(0,1)
peres = c.to_gate(label="Peres")
c.draw()

q_0: ──■────■──
       │  ┌─┴─┐
q_1: ──■──┤ X ├
     ┌─┴─┐└───┘
q_2: ┤ X ├─────
     └───┘

In [269]:
def generate_peres(a,b,c):
    circuit = QuantumCircuit(3, 0)
    if (a == '1'):
        circuit.x(0)
    if (b == '1'):
        circuit.x(1)
    if (c == '1'):
        circuit.x(2)
    circuit.ccx(0,1,2)
    circuit.cx(0,1)
    circuit.measure_all()
    circuit = circuit.reverse_bits()
    return circuit

inputs = ['0', '1']
for i in range(len(inputs)):
    for j in range(len(inputs)):
        for k in range(len(inputs)):
            c = generate_peres(inputs[i], inputs[j], inputs[k])
            usim = Aer.get_backend('unitary_simulator')
            transpiled = transpile(c, backend=usim)
            backend = Aer.get_backend('aer_simulator')
            job = backend.run(transpiled, shots=1, memory=True)
            output = job.result().get_memory()[0]
            print("in: "+inputs[i]+inputs[j]+inputs[k]+" r: "+output)

in: 000 r: 000
in: 001 r: 001
in: 010 r: 010
in: 011 r: 011
in: 100 r: 110
in: 101 r: 111
in: 110 r: 101
in: 111 r: 100


In [232]:
c.draw()

┌───┐┌───┐      ░       ┌─┐
   q_0: ┤ X ├┤ X ├──────░───────┤M├
        ├───┤└─┬─┘┌───┐ ░    ┌─┐└╥┘
   q_1: ┤ X ├──■──┤ X ├─░────┤M├─╫─
        ├───┤  │  └─┬─┘ ░ ┌─┐└╥┘ ║ 
   q_2: ┤ X ├──■────■───░─┤M├─╫──╫─
        └───┘           ░ └╥┘ ║  ║ 
meas: 3/═══════════════════╩══╩══╩═
                           2  1  0

### Full Adder

In [271]:
"""
Input: 
a = 1st operand, 
b = 2nd operand, 
c = 0, 
d = c_in

Output: 
P = garbage, 
Q = garbage, 
R = sum, 
T = carry
"""
c = QuantumCircuit(4)
c.append(peres, [0,1,2])
c.swap(2,3)
c.append(peres, [1,2,3])
#c = c.reverse_bits()
fa = c.to_gate(label="RFA")
c.draw()

┌────────┐             
q_0: ┤0       ├─────────────
     │        │   ┌────────┐
q_1: ┤1 Peres ├───┤0       ├
     │        │   │        │
q_2: ┤2       ├─X─┤1 Peres ├
     └────────┘ │ │        │
q_3: ───────────X─┤2       ├
                  └────────┘

In [273]:
def generate_fa(a,b,d):
    circuit = QuantumCircuit(4, 2)
    if (a == '1'):
        circuit.x(0)
    if (b == '1'):
        circuit.x(1)
    if (d == '1'):
        circuit.x(3)
    circuit.append(peres, [0,1,2])
    circuit.swap(2,3)
    circuit.append(peres, [1,2,3])
    circuit.measure(2,0)
    circuit.measure(3,1)
    #circuit.measure_all()
    circuit = circuit.reverse_bits()
    return circuit

inputs = ['0','1']
for i in range(len(inputs)):
    for j in range(len(inputs)):
        for k in range(len(inputs)):
            c = generate_fa(inputs[i], inputs[j], inputs[k])
            usim = Aer.get_backend('unitary_simulator')
            transpiled = transpile(c, backend=usim)
            backend = Aer.get_backend('aer_simulator')
            job = backend.run(transpiled, shots=1, memory=True)
            output = job.result().get_memory()[0]
            print("in: "+inputs[i]+inputs[j]+inputs[k]+" r: "+output)

in: 000 r: 00
in: 001 r: 10
in: 010 r: 10
in: 011 r: 01
in: 100 r: 10
in: 101 r: 01
in: 110 r: 01
in: 111 r: 11


### Fredkin

In [287]:
"""
By considering only the output qubit Q and by using input qubits 'a' as the selection line, Fredkin gate can 
be used as a 2x1 MUX.

Input: a, b, c

Output:
P = a,
Q = a'*b + a*c
R = a*b + a'*c
"""
c = QuantumCircuit(3)
c.append(revcx, [1,2])
c.ccx(0,1,2)
c.append(revcx, [1,2])
fredkin = c.to_gate(label="FG")
c.draw()

q_0: ───────────────■───────────────
     ┌───────────┐  │  ┌───────────┐
q_1: ┤0          ├──■──┤0          ├
     │  rev_CNOT │┌─┴─┐│  rev_CNOT │
q_2: ┤1          ├┤ X ├┤1          ├
     └───────────┘└───┘└───────────┘

In [288]:
def generate_fg(a,b,c):
    circuit = QuantumCircuit(3, 0)
    if (a == '1'):
        circuit.x(0)
    if (b == '1'):
        circuit.x(1)
    if (c == '1'):
        circuit.x(2)
    circuit.append(revcx, [1,2])
    circuit.ccx(0,1,2)
    circuit.append(revcx, [1,2])
    circuit.measure_all()
    circuit = circuit.reverse_bits()
    return circuit

inputs = ['0', '1']
for i in range(len(inputs)):
    for j in range(len(inputs)):
        for k in range(len(inputs)):
            c = generate_fg(inputs[i], inputs[j], inputs[k])
            usim = Aer.get_backend('unitary_simulator')
            transpiled = transpile(c, backend=usim)
            backend = Aer.get_backend('aer_simulator')
            job = backend.run(transpiled, shots=1, memory=True)
            output = job.result().get_memory()[0]
            print("in: "+inputs[i]+inputs[j]+inputs[k]+" r: "+output)

in: 000 r: 000
in: 001 r: 001
in: 010 r: 010
in: 011 r: 011
in: 100 r: 100
in: 101 r: 110
in: 110 r: 101
in: 111 r: 111


# CSA Design-1

Each stage of the CSA can be designed as follows:

In [275]:
"""
This circuit represents a single stage of the CSA.

Input:
1 = first operand, represents "a"
2 = second operand, represents "b"
3 = 0, ancilla, copy of "a"
4 = 0, ancilla, copy of "b"
5 = 0, represents "c=0"
6 = 1, represents "c=1"
7 = 0, ancilla
8 = 0, ancilla
9 = input carry
10 = 0, ancilla

Output:
1 = g
2 = g
3 = g
4 = g
5 = carry
6 = g
7 = sum
8 = g
9 = carry
10 = g

5 ancilla bits and 8 garbage bits if we include the copy of the carry.
"""

a_in = QuantumRegister(1, name="a")
b_in = QuantumRegister(1, name="b")
a_copies = QuantumRegister(1, name="a'")
b_copies = QuantumRegister(1, name="b'")
carries = QuantumRegister(2, name="c")
zero = QuantumRegister(2, name="zero")
c_in = QuantumRegister(2, name="c_in")

csa = QuantumCircuit(a_in, b_in, a_copies, b_copies, carries, zero, c_in)
csa.reset(a_copies[0])
csa.reset(b_copies[0])
csa.cx(a_in[0], a_copies[0])                                        # a' = a
csa.cx(b_in[0], b_copies[0])                                        # b' = b
csa.reset(carries[0])                                               # c = 0
csa.reset(carries[1])
csa.x(carries[1])                                                   # c = 1
csa.reset(zero[0])
csa.reset(zero[1])
csa.reset(c_in[1])                                                  # c_in' = c_in

csa.cx(c_in[0], c_in[1])
csa.append(fa, [a_in[0], b_in[0], zero[0], carries[0]])
csa.append(fa, [a_copies[0], b_copies[0], zero[1], carries[1]])
csa.append(fredkin, [c_in[1], zero[0], zero[1]])                    # mux for the sum qubits
csa.append(fredkin, [c_in[0], carries[0], carries[1]])              # mux for the carry qubits

csa.reset(c_in[0])                                                  # Recycle c_in for the output of buffers

csa.cx(carries[0], c_in[0])                                         # Buffer

csa.draw()

┌──────┐                                
     a: ───────■───────┤0     ├────────────────────────────────
               │       │      │                                
     b: ───────┼────■──┤1     ├────────────────────────────────
             ┌─┴─┐  │  │      │┌──────┐                        
    a': ─|0>─┤ X ├──┼──┤      ├┤0     ├────────────────────────
             └───┘┌─┴─┐│      ││      │                        
    b': ─|0>──────┤ X ├┤  RFA ├┤1     ├────────────────────────
                  └───┘│      ││      │       ┌─────┐          
   c_0: ─|0>───────────┤3     ├┤      ├───────┤1    ├───────■──
             ┌───┐     │      ││  RFA │       │     │       │  
   c_1: ─|0>─┤ X ├─────┤      ├┤3     ├───────┤2    ├───────┼──
             └───┘     │      ││      │┌─────┐│     │       │  
zero_0: ─|0>───────────┤2     ├┤      ├┤1    ├┤  FG ├───────┼──
                       └──────┘│      ││     ││     │       │  
zero_1: ─|0>───────────────────┤2     ├┤2    ├┤     ├───────┼──
                               └──────┘│  FG ││     │     ┌─┴─┐
c_in_0: ───────■───────────────────────┤     ├┤0    ├─|0>─┤ X ├
             ┌─┴─┐                     │     │└─────┘     └───┘
c_in_1: ─|0>─┤ X ├─────────────────────┤0    ├─────────────────
             └───┘                     └─────┘

Note that although the logic gates are represented in series, the circuit will be re-arranged by the transpiler in order to execute gates in parallel where possible. For instance, if you swap the position of "b" with "a'" it is clear that the two CNOTs at the beginning of the circuit can be run simultaneously.

The quantum logic gate CSA has number of input and output of 10 and we have different options to combine the stages together:

A first approach is by remarking the classical CSA exploiting the fact that all the FA operations can be done in parallel, but that means using a lot of qubits. As a matter of fact, if we want to make a two stage CSA, we have to connect the input qubits "i" and "l" (the carries propagated by the first stage) of the second CSA to the output T and X of the first one. To force the parallel execution of the four RFAs instead, we have to use 8 qubits for the first stage and 8 more qubits for the second stage. Therefore the total number of qubits used to implement the circuit is 8n+2. 

The number of garbage bits is given by 8n+2-(n+1) = 7n+1, where n+1 is given by: "n" for the number of bits representing the result and 1 for the carry out. Ancilla bits are instead 5n, because we have n copy of "a", n copy of "b", 2n copy zeros and n copies of input carries.

Another approach is to run each stage in series, reducing the number of qubits used to 10 (fixed, regardless of the number of stages), but in this way the CSA looses all its advantages because the delay grows with the number of stages.

## Iterative construction of Design-1

Based on the discussion above, here's an iterative fucntion to build CSAs operating on n-qubits numbers. Measurments and output registers to store classical bits are added to test the circuit.

In [318]:
def generate_CSA1_n_qubits(n, a, b, c):
    if len(a) != n or len(b) != n:
        return "Length of inputs differs from length of n."
    a = a[::-1]
    b = b[::-1]

    a_in = QuantumRegister(n, name="a")
    b_in = QuantumRegister(n, name="b")
    a_copies = QuantumRegister(n, name="a'")
    b_copies = QuantumRegister(n, name="b'")
    carries = QuantumRegister(n, name="c")
    carries_copies = QuantumRegister(n, name="c'")
    zero = QuantumRegister(n, name="zero")
    zero_copies = QuantumRegister(n, name="zero'")
    c_in = QuantumRegister(2, name="c_in")

    output = ClassicalRegister(n+1, name='output')

    csa = QuantumCircuit(a_in, b_in, a_copies, b_copies, carries, carries_copies, zero, zero_copies, c_in, output)

    if (c == '1'):
        csa.x(c_in[0])
    for i in range(n):
        if (a[i] == '1'):
            csa.x(a_in[i])
        if (b[i] == '1'):
            csa.x(b_in[i]) 
        csa.reset(a_copies[i])
        csa.reset(b_copies[i])
        csa.cx(a_in[i], a_copies[i])                                                        # a' = a
        csa.cx(b_in[i], b_copies[i])                                                        # b' = b
        csa.reset(carries[i])                                                               # c = 0
        csa.reset(carries_copies[i])
        csa.x(carries_copies[i])                                                            # c = 1
        csa.reset(zero[i])
        csa.reset(zero_copies[i])
    csa.reset(c_in[1])                                                                      # c_in' = c_in
    csa.cx(c_in[0], c_in[1])
    
    for i in range(n):
        csa.append(fa, [a_in[i], b_in[i], zero[i], carries[i]])
        csa.append(fa, [a_copies[i], b_copies[i], zero_copies[i], carries_copies[i]])
        if i == 0:
            csa.append(fredkin, [c_in[1], zero[i], zero_copies[i]])                         # mux for the sum qubits
        else:
            csa.append(fredkin, [carries[i-1], zero[i], zero_copies[i]])
        csa.append(fredkin, [c_in[0], carries[i], carries_copies[i]])                       # mux for the carry qubits

        csa.measure(zero[i], output[i])

        if (i < n-1):
            csa.reset(c_in[0])                                                              # Recycle c_in for the output of buffers
            csa.cx(carries[i], c_in[0])                                                     # Buffer
        else:
            csa.measure(carries[i], output[i+1])                                            # Measure the carry out if it's the last stage
    return csa

This is how a two-qubit CSA looks like:

In [346]:
csa2q = generate_CSA1_n_qubits(2, '00', '00', '0')
csa2q.draw()

┌──────┐                               »
     a_0: ───────■─────────────────┤0     ├───────────────────────────────»
                 │                 │      │        ┌──────┐               »
     a_1: ───────┼─────────■───────┤      ├────────┤0     ├───────────────»
                 │         │       │      │        │      │               »
     b_0: ───────┼────■────┼───────┤1     ├────────┤      ├───────────────»
                 │    │    │       │      │        │      │               »
     b_1: ───────┼────┼────┼────■──┤      ├────────┤1     ├───────────────»
               ┌─┴─┐  │    │    │  │      │┌──────┐│      │               »
    a'_0: ─|0>─┤ X ├──┼────┼────┼──┤      ├┤0     ├┤      ├───────────────»
               └───┘  │  ┌─┴─┐  │  │      ││      ││      │┌──────┐       »
    a'_1: ─|0>────────┼──┤ X ├──┼──┤      ├┤      ├┤      ├┤0     ├───────»
                    ┌─┴─┐└───┘  │  │      ││      ││      ││      │       »
    b'_0: ─|0>──────┤ X ├───────┼──┤  RFA ├┤1     ├┤      ├┤      ├───────»
                    └───┘     ┌─┴─┐│      ││      ││      ││      │       »
    b'_1: ─|0>────────────────┤ X ├┤      ├┤      ├┤  RFA ├┤1     ├───────»
                              └───┘│      ││      ││      ││      │       »
     c_0: ─|0>─────────────────────┤3     ├┤      ├┤      ├┤      ├───────»
                                   │      ││      ││      ││      │       »
     c_1: ─|0>─────────────────────┤      ├┤  RFA ├┤3     ├┤      ├───────»
               ┌───┐               │      ││      ││      ││      │       »
    c'_0: ─|0>─┤ X ├───────────────┤      ├┤3     ├┤      ├┤  RFA ├───────»
               ├───┤               │      ││      ││      ││      │       »
    c'_1: ─|0>─┤ X ├───────────────┤      ├┤      ├┤      ├┤3     ├───────»
               └───┘               │      ││      ││      ││      │┌─────┐»
  zero_0: ─|0>─────────────────────┤2     ├┤      ├┤      ├┤      ├┤1    ├»
                                   └──────┘│      ││      ││      ││     │»
  zero_1: ─|0>─────────────────────────────┤      ├┤2     ├┤      ├┤     ├»
                                           │      │└──────┘│      ││     │»
 zero'_0: ─|0>─────────────────────────────┤2     ├────────┤      ├┤2    ├»
                                           └──────┘        │      ││  FG │»
 zero'_1: ─|0>─────────────────────────────────────────────┤2     ├┤     ├»
                                                           └──────┘│     │»
  c_in_0: ───────■─────────────────────────────────────────────────┤     ├»
               ┌─┴─┐                                               │     │»
  c_in_1: ─|0>─┤ X ├───────────────────────────────────────────────┤0    ├»
               └───┘                                               └─────┘»
output: 3/════════════════════════════════════════════════════════════════»
                                                                          »
«                                                  
«     a_0: ────────────────────────────────────────
«                                                  
«     a_1: ────────────────────────────────────────
«                                                  
«     b_0: ────────────────────────────────────────
«                                                  
«     b_1: ────────────────────────────────────────
«                                                  
«    a'_0: ────────────────────────────────────────
«                                                  
«    a'_1: ────────────────────────────────────────
«                                                  
«    b'_0: ────────────────────────────────────────
«                                                  
«    b'_1: ────────────────────────────────────────
«          ┌─────┐             ┌─────┐             
«     c_0: ┤1    ├──────────■──┤0    ├─────────────
«          │     │          │  │     │┌─────┐   ┌─┐
«     c_1: ┤     ├──────────┼──┤     ├┤1    ├───┤M├
«          │     │          │  

Now I am going to test if the CSA exactly reproduce the truth table of a classical one-bit CSA, that is:

    a   b   c   |   out

    --------------------

    0   0   0   |   00

    0   0   1   |   01

    0   1   0   |   01

    0   1   1   |   10

    1   0   0   |   01

    1   0   1   |   10

    1   1   0   |   10

    1   1   1   |   11

Where the MSB of "out" is the carry out and the LSB is the sum. As you can see from the result below, the CSA works properly:
    

In [320]:
inputs = ['0', '1']
for i in range(len(inputs)):
    for j in range(len(inputs)):
        for k in range(len(inputs)):
            csa2q = generate_CSA1_n_qubits(1, inputs[i], inputs[j], inputs[k])
            usim = Aer.get_backend('unitary_simulator')
            transpiled = transpile(csa2q, backend=usim)
            backend = Aer.get_backend('aer_simulator')
            job = backend.run(transpiled, shots=1, memory=True)
            output = job.result().get_memory()[0]
            print("a: "+inputs[i]+" b: "+inputs[j]+" c_in: "+inputs[k]+" r: "+output)

a: 0 b: 0 c_in: 0 r: 00
a: 0 b: 0 c_in: 1 r: 01
a: 0 b: 1 c_in: 0 r: 01
a: 0 b: 1 c_in: 1 r: 10
a: 1 b: 0 c_in: 0 r: 01
a: 1 b: 0 c_in: 1 r: 10
a: 1 b: 1 c_in: 0 r: 10
a: 1 b: 1 c_in: 1 r: 11


Let's test now the two-qubit CSA and even this one works as intended:

In [321]:
inputs = ['00', '01', '10', '11']
carries = ['0', '1']
for i in range(len(inputs)):
    for j in range(len(inputs)):
        csa2q = generate_CSA1_n_qubits(2, inputs[i], inputs[j], '0')
        usim = Aer.get_backend('unitary_simulator')
        transpiled = transpile(csa2q, backend=usim)
        backend = Aer.get_backend('aer_simulator')
        job = backend.run(transpiled, shots=1, memory=True)
        output = job.result().get_memory()[0]
        print("a: "+inputs[i]+" b: "+inputs[j]+" r: "+output)

a: 00 b: 00 r: 000
a: 00 b: 01 r: 001
a: 00 b: 10 r: 010
a: 00 b: 11 r: 011
a: 01 b: 00 r: 001
a: 01 b: 01 r: 010
a: 01 b: 10 r: 011
a: 01 b: 11 r: 100
a: 10 b: 00 r: 010
a: 10 b: 01 r: 011
a: 10 b: 10 r: 100
a: 10 b: 11 r: 101
a: 11 b: 00 r: 011
a: 11 b: 01 r: 100
a: 11 b: 10 r: 101
a: 11 b: 11 r: 110


# CSA Design-2

In [330]:
"""
This circuit represents a single stage of the CSA.

Input:
1 = first operand, represents "a"
2 = second operand, represents "b"
11 = input carry

Initialization of other states (ancilla):
3 = copy of a
4 = copy of b
5 = second copy of b
6 = not a
7 = not b
8 = copy of not a
9 = 0
10 = 1
12 = copy of input carry

Output:
1 = g
2 = sum
3 = g
4 = g
5 = g
6 = g
7 = copy of carry out
8 = g
9 = carry out
10 = g
11 = g
12 = g

9 ancilla bits and 9 garbage bits.
"""

a_in = QuantumRegister(1, name="a")
b_in = QuantumRegister(1, name="b")
a_copies = QuantumRegister(1, name="a'")
b_copies = QuantumRegister(1, name="b'")
b_copies2 = QuantumRegister(1, name="b''")
a_not = QuantumRegister(1, name="a_not")
b_not = QuantumRegister(1, name="b_not")
a_not_copies = QuantumRegister(1, name="a_not'")
zero = QuantumRegister(1, name="zero")
one = QuantumRegister(1, name="one")
c_in = QuantumRegister(2, name="c_in")

csa2 = QuantumCircuit(a_in, b_in, a_copies, b_copies, b_copies2, a_not, b_not, a_not_copies, zero, one, c_in)

# Initialization of qubit states
csa2.reset(a_copies[0])
csa2.reset(b_copies[0])
csa2.reset(b_copies2[0])
csa2.cx(a_in[0], a_copies[0])
csa2.cx(b_in[0], b_copies[0])
csa2.cx(b_in[0], b_copies2[0])
csa2.reset(a_not[0])
csa2.cx(a_in[0], a_not[0])
csa2.reset(b_not[0])
csa2.cx(b_in[0], b_not[0])
csa2.x(a_not[0])
csa2.x(b_not[0])
csa2.reset(a_not_copies[0])
csa2.cx(a_not[0], a_not_copies[0])
csa2.reset(zero[0])
csa2.reset(one[0])
csa2.x(one[0])
csa2.reset(c_in[0])
csa2.reset(c_in[1])
csa2.cx(c_in[0], c_in[1])

# Circuit
csa2.barrier()
csa2.cx(a_in[0], b_in[0])
csa2.cx(a_not[0], b_copies[0])
csa2.append(fredkin, [c_in[0], b_in[0], b_copies[0]])               # Sum is on b_in[0]

csa2.append(peres, [a_copies[0], b_copies2[0], zero[0]])            # Carry 1 is on zero[0]
csa2.append(peres, [a_not_copies[0], b_not[0], one[0]])             # Carry 2 is on one[0]
csa2.append(fredkin, [c_in[1], zero[0], one[0]])                    # Carry out is on zero[0]

csa2.reset(b_not[0])                                                # Recycle a garbage bit generated by Peres gate
csa2.cx(zero[0], b_not[0])                                          # A copy of the carry out is in b_not[0]

csa2.draw()

░                     »
       a: ───────■──────────────■──────────────────░─────■───────────────»
                 │              │                  ░   ┌─┴─┐             »
       b: ───────┼────■────■────┼────■─────────────░───┤ X ├─────────────»
               ┌─┴─┐  │    │    │    │             ░   └───┘   ┌────────┐»
      a': ─|0>─┤ X ├──┼────┼────┼────┼─────────────░───────────┤0       ├»
               └───┘┌─┴─┐  │    │    │             ░   ┌───┐   │        │»
      b': ─|0>──────┤ X ├──┼────┼────┼─────────────░───┤ X ├───┤        ├»
                    └───┘┌─┴─┐  │    │             ░   └─┬─┘   │        │»
     b'': ─|0>───────────┤ X ├──┼────┼─────────────░─────┼─────┤1       ├»
                         └───┘┌─┴─┐  │  ┌───┐      ░     │     │        │»
   a_not: ─|0>────────────────┤ X ├──┼──┤ X ├──■───░─────■─────┤  Peres ├»
                              └───┘┌─┴─┐├───┤  │   ░ ┌────────┐│        │»
   b_not: ─|0>─────────────────────┤ X ├┤ X ├──┼───░─┤1       ├┤        ├»
                                   └───┘└───┘┌─┴─┐ ░ │        ││        │»
  a_not': ─|0>───────────────────────────────┤ X ├─░─┤0       ├┤        ├»
                                             └───┘ ░ │  Peres ││        │»
    zero: ─|0>─────────────────────────────────────░─┤        ├┤2       ├»
               ┌───┐                               ░ │        │└────────┘»
     one: ─|0>─┤ X ├───────────────────────────────░─┤2       ├──────────»
               └───┘                               ░ └────────┘          »
  c_in_0: ─|0>───■─────────────────────────────────░─────────────────────»
               ┌─┴─┐                               ░                     »
  c_in_1: ─|0>─┤ X ├───────────────────────────────░─────────────────────»
               └───┘                               ░                     »
output: 2/═══════════════════════════════════════════════════════════════»
                                                                         »
«                             
«       a: ───────────────────
«          ┌─────┐            
«       b: ┤1    ├────────────
«          │     │            
«      a': ┤     ├────────────
«          │     │            
«      b': ┤2    ├────────────
«          │     │            
«     b'': ┤     ├────────────
«          │     │            
«   a_not: ┤     ├────────────
«          │  FG │       ┌───┐
«   b_not: ┤     ├──|0>──┤ X ├
«          │     │       └─┬─┘
«  a_not': ┤     ├─────────┼──
«          │     │┌─────┐  │  
«    zero: ┤     ├┤1    ├──■──
«          │     ││     │     
«     one: ┤     ├┤2    ├─────
«          │     ││  FG │     
«  c_in_0: ┤0    ├┤     ├─────
«          └─────┘│     │     
«  c_in_1: ───────┤0    ├─────
«                 └─────┘     
«output: 2/═══════════════════
«

In [335]:
def generate_CSA2_n_qubits(n, a, b, c):
    if len(a) != n or len(b) != n:
        return "Length of inputs differs from length of n."
    a = a[::-1]
    b = b[::-1]
    
    a_in = QuantumRegister(n, name="a")
    b_in = QuantumRegister(n, name="b")
    a_copies = QuantumRegister(n, name="a'")
    b_copies = QuantumRegister(n, name="b'")
    b_copies2 = QuantumRegister(n, name="b''")
    a_not = QuantumRegister(n, name="a_not")
    b_not = QuantumRegister(n, name="b_not")
    a_not_copies = QuantumRegister(n, name="a_not'")
    zero = QuantumRegister(n, name="zero")
    one = QuantumRegister(n, name="one")
    c_in = QuantumRegister(2, name="c_in")
    output = ClassicalRegister(n+1, name="output")

    csa2 = QuantumCircuit(a_in, b_in, a_copies, b_copies, b_copies2, a_not, b_not, a_not_copies, zero, one, c_in, output)

    if (c == '1'):
        csa2.x(c_in[0])
    for i in range(n):
        # Initialization of qubit states
        if (a[i] == '1'):
            csa2.x(a_in[i])
        if (b[i] == '1'):
            csa2.x(b_in[i])
        csa2.reset(a_copies[i])
        csa2.reset(b_copies[i])
        csa2.reset(b_copies2[i])
        csa2.cx(a_in[i], a_copies[i])
        csa2.cx(b_in[i], b_copies[i])
        csa2.cx(b_in[i], b_copies2[i])
        csa2.reset(a_not[i])
        csa2.cx(a_in[i], a_not[i])
        csa2.reset(b_not[i])
        csa2.cx(b_in[i], b_not[i])
        csa2.x(a_not[i])
        csa2.x(b_not[i])
        csa2.reset(a_not_copies[i])
        csa2.cx(a_not[i], a_not_copies[i])
        csa2.reset(zero[i])
        csa2.reset(one[i])
        csa2.x(one[i])
    csa2.reset(c_in[1])
    csa2.cx(c_in[0], c_in[1])
    csa2.barrier()
        
    for i in range(n):
        # Circuit
        csa2.cx(a_in[i], b_in[i])
        csa2.cx(a_not[i], b_copies[i])
        if i == 0:
            csa2.append(fredkin, [c_in[0], b_in[i], b_copies[i]])               # Sum is on b_in[0]
        else:
            csa2.append(fredkin, [zero[i-1], b_in[i], b_copies[i]])
        
        csa2.measure(b_in[i], output[i])

        csa2.append(peres, [a_copies[i], b_copies2[i], zero[i]])                # Carry 1 is on zero[0]
        csa2.append(peres, [a_not_copies[i], b_not[i], one[i]])                 # Carry 2 is on one[0]
        if i == 0:
            csa2.append(fredkin, [c_in[1], zero[i], one[i]])                    # Carry is on zero[0]
        else:
            csa2.append(fredkin, [b_not[i-1], zero[i], one[i]])

        if (i < n-1):
            csa2.reset(b_not[i])                                                # Recycle a garbage bit generated by Peres gate
            csa2.cx(zero[i], b_not[i])                                          # A copy of the carry is in b_not[0]
    csa2.measure(zero[i], output[i+1])
    return csa2

In [339]:
inputs = ['0', '1']
for i in range(len(inputs)):
    for j in range(len(inputs)):
        for k in range(len(inputs)):
            csa2 = generate_CSA2_n_qubits(1, inputs[i], inputs[j], inputs[k])
            usim = Aer.get_backend('unitary_simulator')
            transpiled = transpile(csa2, backend=usim)
            backend = Aer.get_backend('aer_simulator')
            job = backend.run(transpiled, shots=1, memory=True)
            output = job.result().get_memory()[0]
            print("a: "+inputs[i]+" b: "+inputs[j]+" c_in: "+inputs[k]+" r: "+output)

a: 0 b: 0 c_in: 0 r: 00
a: 0 b: 0 c_in: 1 r: 01
a: 0 b: 1 c_in: 0 r: 01
a: 0 b: 1 c_in: 1 r: 10
a: 1 b: 0 c_in: 0 r: 01
a: 1 b: 0 c_in: 1 r: 10
a: 1 b: 1 c_in: 0 r: 10
a: 1 b: 1 c_in: 1 r: 11


In [344]:
inputs = ['00', '01', '10', '11']
carries = ['0', '1']
for i in range(len(inputs)):
    for j in range(len(inputs)):
        csa2 = generate_CSA2_n_qubits(2, inputs[i], inputs[j], '0')
        usim = Aer.get_backend('unitary_simulator')
        transpiled = transpile(csa2, backend=usim)
        backend = Aer.get_backend('aer_simulator')
        job = backend.run(transpiled, shots=1, memory=True)
        output = job.result().get_memory()[0]
        print("a: "+inputs[i]+" b: "+inputs[j]+" r: "+output)

a: 00 b: 00 r: 000
a: 00 b: 01 r: 001
a: 00 b: 10 r: 010
a: 00 b: 11 r: 011
a: 01 b: 00 r: 001
a: 01 b: 01 r: 010
a: 01 b: 10 r: 011
a: 01 b: 11 r: 100
a: 10 b: 00 r: 010
a: 10 b: 01 r: 011
a: 10 b: 10 r: 100
a: 10 b: 11 r: 101
a: 11 b: 00 r: 011
a: 11 b: 01 r: 100
a: 11 b: 10 r: 101
a: 11 b: 11 r: 110


In [345]:
csa2.draw()

┌───┐                                                            »
     a_0: ┤ X ├──■────────────────────────■────────────────────────────────»
          ├───┤  │                        │                                »
     a_1: ┤ X ├──┼─────────■──────────────┼─────────■──────────────────────»
          ├───┤  │         │              │         │                      »
     b_0: ┤ X ├──┼────■────┼─────────■────┼─────────┼────■─────────────────»
          ├───┤  │    │    │         │    │         │    │                 »
     b_1: ┤ X ├──┼────┼────┼────■────┼────┼────■────┼────┼────■────────────»
          └───┘┌─┴─┐  │    │    │    │    │    │    │    │    │            »
    a'_0: ─|0>─┤ X ├──┼────┼────┼────┼────┼────┼────┼────┼────┼────────────»
               └───┘  │  ┌─┴─┐  │    │    │    │    │    │    │            »
    a'_1: ─|0>────────┼──┤ X ├──┼────┼────┼────┼────┼────┼────┼────────────»
                    ┌─┴─┐└───┘  │    │    │    │    │    │    │            »
    b'_0: ─|0>──────┤ X ├───────┼────┼────┼────┼────┼────┼────┼────────────»
                    └───┘     ┌─┴─┐  │    │    │    │    │    │            »
    b'_1: ─|0>────────────────┤ X ├──┼────┼────┼────┼────┼────┼────────────»
                              └───┘┌─┴─┐  │    │    │    │    │            »
   b''_0: ─|0>─────────────────────┤ X ├──┼────┼────┼────┼────┼────────────»
                                   └───┘  │  ┌─┴─┐  │    │    │            »
   b''_1: ─|0>────────────────────────────┼──┤ X ├──┼────┼────┼────────────»
                                        ┌─┴─┐├───┤  │    │    │            »
 a_not_0: ─|0>──────────────────────────┤ X ├┤ X ├──┼────┼────┼─────────■──»
                                        └───┘└───┘┌─┴─┐  │    │  ┌───┐  │  »
 a_not_1: ─|0>────────────────────────────────────┤ X ├──┼────┼──┤ X ├──┼──»
                                                  └───┘┌─┴─┐  │  ├───┤  │  »
 b_not_0: ─|0>─────────────────────────────────────────┤ X ├──┼──┤ X ├──┼──»
                                                       └───┘┌─┴─┐├───┤  │  »
 b_not_1: ─|0>──────────────────────────────────────────────┤ X ├┤ X ├──┼──»
                                                            └───┘└───┘┌─┴─┐»
a_not'_0: ─|0>────────────────────────────────────────────────────────┤ X ├»
                                                                      └───┘»
a_not'_1: ─|0>─────────────────────────────────────────────────────────────»
                                                                           »
  zero_0: ─|0>─────────────────────────────────────────────────────────────»
                                                                           »
  zero_1: ─|0>─────────────────────────────────────────────────────────────»
               ┌───┐                                                       »
   one_0: ─|0>─┤ X ├───────────────────────────────────────────────────────»
               ├───┤                                                       »
   one_1: ─|0>─┤ X ├───────────────────────────────────────────────────────»
               └───┘                                                       »
  c_in_0: ───────■─────────────────────────────────────────────────────────»
               ┌─┴─┐                                                       »
  c_in_1: ─|0>─┤ X ├───────────────────────────────────────────────────────»
               └───┘                                                       »
output: 3/═════════════════════════════════════════════════════════════════»
                                                                           »
«                ░                                                          »
«     a_0: ──────░─────■────────────────────────────────────────────────────»
«                ░     │                                                    »
«     a_1: ──────░─────┼─────────■──────────────────────────────────────────»
«                ░   ┌─┴─┐       │                         ┌─────┐       ┌─┐»
«

## Two-pair Two-rail checker